In [ ]:
!pip install split-folders

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import glob
import PIL
import pathlib
import splitfolders
import random
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras import regularizers
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
splitfolders.ratio("/kaggle/input/thesisdataset/thesdats",
                   output="/kaggle/working/dataset_split",
                   ratio=(0.7, 0.15, 0.15))

In [ ]:
data_dir_train = pathlib.Path( '/kaggle/working/dataset_split/train')
img_height,  img_width  = 299, 299

In [ ]:
train_ds = image_dataset_from_directory(data_dir_train,
                                        seed = 123,
                                        image_size=(img_height, img_width))

In [ ]:
class_names = train_ds.class_names

In [ ]:
class_names

In [ ]:
### Visualizing the training data
plt.figure(figsize=(15, 10))

for i, class_ in enumerate(list(class_names)):
    plt.subplot(3, 3, i+1)

    # Correct indentation for the following lines
    data_path = os.path.join(str(data_dir_train), class_)
    file_paths = glob.glob(os.path.join(data_path, '*.*'))
    random_img_path = random.choice(file_paths)
    img = PIL.Image.open(random_img_path)
    plt.imshow(img)
    plt.title(class_)
    plt.axis("off")

plt.show()

In [ ]:
class_size = {}
for name in class_names:
    class_size[name] = len(list(data_dir_train.glob(name+'/*.*')))

class_size

In [ ]:
class_df = pd.DataFrame(class_size.items(),index=list(class_size), columns = ['ClassName', 'NumberOfSamples'])
class_df.drop(['ClassName'], axis = 1, inplace=True)
class_df

# **Augmentation**

In [ ]:
!pip install Augmentor

In [ ]:
import Augmentor

In [ ]:

path_to_training_dataset = '/kaggle/working/dataset_split/train/'
target_samples = 1500  # Desired number of samples per class

for i in class_names:
    class_path = os.path.join(path_to_training_dataset, i)
    existing_samples = len(os.listdir(class_path))  # Count existing images
    additional_samples = max(0, target_samples - existing_samples)  # Calculate needed samples

    if additional_samples > 0:
        p = Augmentor.Pipeline(class_path, output_directory='')

        # 1. Extreme Rotation
        p.rotate(probability=0.9, max_left_rotation=25, max_right_rotation=25)

        # 2. Horizontal & Vertical Flip
        p.flip_left_right(probability=0.8)
        p.flip_top_bottom(probability=0.5)

        # 3. Random Zoom
        p.zoom_random(probability=0.7, percentage_area=0.5)

        # 4. Random Brightness
        p.random_brightness(probability=0.8, min_factor=0.4, max_factor=1.8)

        # 5. Random Contrast
        p.random_contrast(probability=0.7, min_factor=0.3, max_factor=2.0)

        # 6. Random Distortions (Mimics warping)
        p.random_distortion(probability=0.7, grid_width=6, grid_height=6, magnitude=12)

        # 7. Shearing
        p.shear(probability=0.6, max_shear_left=20, max_shear_right=20)

        # 8. Skewing
        p.skew(probability=0.6, magnitude=0.5)

        # 9. Gaussian Noise
        p.gaussian_distortion(probability=0.6, grid_width=5, grid_height=5, magnitude=8, corner='bell', method='in')

        # 10. Greyscale Conversion (Black & White Effect)
        p.greyscale(probability=0.4)

        # 11. Inversion (Negative Effect)
        p.invert(probability=0.3)

        # 12. Crop & Resize (For additional variation)
        p.crop_random(probability=0.5, percentage_area=0.7)
        p.resize(probability=1.0, width=128, height=128)

      

        # Generate only the necessary number of images
        p.sample(additional_samples)


# **Model**

In [ ]:
batch_size = 32
epochs = 50
img_size = 224

In [ ]:
train_datagen = ImageDataGenerator( rescale=1./255,
                                  rotation_range=5,  # rotation
                                   width_shift_range=0.2,  # horizontal shift
                                   zoom_range=0.2,  # zoom
                                   horizontal_flip=True,
                               # horizontal flip
                                   brightness_range=[0.2,0.8])

test_datagen = ImageDataGenerator(
                                   rescale=1./255,
                                   rotation_range=5,  # rotation
                                   width_shift_range=0.2,  # horizontal shift
                                   zoom_range=0.2,  # zoom
                                   horizontal_flip=True,  # horizontal flip
                                   brightness_range=[0.2,0.8])
training_set = train_datagen.flow_from_directory('/kaggle/working/dataset_split/train',
                                               target_size=(224,224),
                                                 batch_size=32)

test_set = test_datagen.flow_from_directory('/kaggle/working/dataset_split/val',
                                            target_size=(224,224),
                                            batch_size=32,
                                            shuffle=False)

In [ ]:
# Focal Loss Definition (better for imbalanced data)

import tensorflow.keras.backend as K
import tensorflow as tf
# Update focal_loss to point to the function focal_loss_fixed
def focal_loss(gamma=2., alpha=[0.25, 0.40, 0.20, 0.15, 0.20]):
    # Convert alpha to a tensor for computation
    alpha = K.constant(alpha)

    def focal_loss_fixed(y_true, y_pred):
        # Clip predictions to prevent log(0)
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())

        # Calculate cross-entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Compute weights for each class
        weight = alpha * K.pow(1 - y_pred, gamma)

        # Apply the weights to the cross-entropy
        loss = K.sum(weight * cross_entropy, axis=-1)

        # Return mean loss across the batch (for better stability during training)
        return K.mean(loss)

    return focal_loss_fixed


In [ ]:
def Xception():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 299x299x3
        input_shape = (img_size , img_size , 3),

    )


    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.75)(x)
    x = tf.keras.layers.BatchNormalization(
                      axis=-1,
                      momentum=0.99,
                      epsilon=0.01,
                      center=True,
                      scale=True,
                      beta_initializer="zeros",
                      gamma_initializer="ones",
                      moving_mean_initializer="zeros",
                      moving_variance_initializer="ones",
                      beta_regularizer=None,
                      gamma_regularizer=None,
                      beta_constraint=None,
                      gamma_constraint=None,

                  )(x)

    out = tf.keras.layers.Dense(5,
                                activation = 'softmax',
                                kernel_regularizer=regularizers.l2(0.02) ,
                                name = 'dense_output'
                                )(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        # Set loss to focal loss
        loss=focal_loss(gamma=2.0, alpha=[0.25, 0.40, 0.20, 0.15, 0.20]),
        # Set metrics to accuracy
        metrics = ['accuracy']
    )

    return model

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                            patience=3,
                                                            verbose=2,
                                                            factor=0.5,
                                                            min_lr=0.00001)
reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=4, min_lr=0.00001)
     

In [ ]:
filepath = '/kaggle/working/checkpoint/mymodel-best.keras'
checkpoint = ModelCheckpoint(filepath , save_best_only= True, monitor = 'val_accuracy')

In [ ]:
import os
os.makedirs('/kaggle/working/Models/categories/', exist_ok=True)

In [ ]:
import numpy as np
import time

def train():
    time_start = time.time()

    model = Xception()

    model.summary()

    
    history = model.fit(
        training_set,
        epochs=50,
        validation_data=test_set,
        callbacks=[early_stopping , learning_rate_reduction, checkpoint]
    )

    model.save_weights('/kaggle/working/Models/categories/category.weights.h5')
    model.save('/kaggle/working/Models/categories/category.h5')

    print('Model saved.')

    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return history


In [ ]:
def test():
    #test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = keras.models.load_model('/kaggle/working/Models/categories/category.h5', custom_objects={'focal_loss_fixed': focal_loss()})
    mod.evaluate(test_set)

    prob = mod.predict(test_set)
    predIdxs = np.argmax(prob, axis=1)


    print('\n')
    print(classification_report(test_set.labels, predIdxs,target_names = key, digits=5))
    return  prob, predIdxs, mod

In [ ]:
input_path = '/kaggle/working/dataset_split/val'
train_data = image_dataset_from_directory(directory=input_path,
                                              batch_size=32,
                                              image_size=(299, 299))

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    plt.figure(figsize=(12, 5))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.show()

if __name__ == "__main__":
    key = train_data.class_names
    train_history = train()
    prob, predIdxs, model = test()

    # Plot training history
    plot_training_history(train_history)

In [ ]:
# Define test data generator for final testing
final_test_datagen = ImageDataGenerator(rescale=1./255)

# Load final test set
final_test_set = final_test_datagen.flow_from_directory('/kaggle/working/dataset_split/test/',
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        shuffle=False)  

# Load the model
mod = keras.models.load_model('/kaggle/working/Models/categories/category.h5', custom_objects={'focal_loss_fixed': focal_loss()})
 
# Evaluate the model on the final test set
final_test_loss, final_test_accuracy = mod.evaluate(final_test_set)
print(f'Final Test Loss: {final_test_loss}')
print(f'Final Test Accuracy: {final_test_accuracy}')

# Predicting and classification report
prob = model.predict(final_test_set)
predIdxs = np.argmax(prob, axis=1)

# Get class names from class indices
class_names = list(final_test_set.class_indices.keys())

# Print classification report
from sklearn.metrics import classification_report
print(classification_report(final_test_set.classes, predIdxs, target_names=class_names))

In [ ]:
 #Confusion Matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Generate confusion matrix
conf_matrix = confusion_matrix(final_test_set.classes, predIdxs)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='RdPu', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()